In [1]:
import logging
import warnings

from pyspark.sql import SparkSession
from classify_amazonreviews_sparkml import run_pipeline

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
    .master("local[4]") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

start_date = '2014-01-01'
split_date = '2014-12-01'

run_pipeline(spark, start_date, split_date)

22/11/11 19:22:57 WARN Utils: Your hostname, MacBook-Pro-4.local resolves to a loopback address: 127.0.0.1; using 192.168.178.19 instead (on interface en0)
22/11/11 19:22:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/11 19:22:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Accuracy: 0.8175150408977219


In [2]:
from freamon.adapters.pyspark.provenance import from_trace

view_gen = from_trace()

INFO:root:Registering source 0 with columns: ['_c0', 'marketplace', 'customer_id', 'review_id', 'product_id', 'vine', 'third_party', 'review_headline', 'review_body', 'review_date', 'prov_id_source_0']
INFO:root:Registering source 1 with columns: ['_c0', 'product_id', 'product_parent', 'product_title', 'category_id', 'prov_id_source_1']
INFO:root:Registering source 2 with columns: ['_c0', 'id', 'category', 'prov_id_source_2']
INFO:root:Registering source 3 with columns: ['_c0', 'review_id', 'star_rating', 'helpful_votes', 'total_votes', 'prov_id_source_3']
INFO:root:Computing train features and predictions...
INFO:root:Computing test features and predictions...                            


In [4]:
materialized_view = view_gen.test_view(
    sliceable_by=['category', 'star_rating'], 
    with_features=False, 
    with_y_true=True, 
    with_y_pred=True)

materialized_view

,category,star_rating,y_true,y_pred
0,Digital_Software,5,0,0.0
1,Digital_Software,4,0,0.0
2,Digital_Software,5,0,0.0
3,Digital_Video_Games,5,0,0.0
4,Digital_Software,5,0,0.0
...,...,...,...,...
29581,Digital_Video_Games,3,0,0.0
29582,Digital_Video_Games,5,1,0.0
29583,Digital_Software,3,1,1.0
29584,Digital_Video_Games,3,0,0.0


In [5]:
from fairlearn.metrics import MetricFrame, false_positive_rate
from sklearn.metrics import recall_score

fairness_metrics = MetricFrame(
    metrics={ 'tpr' : recall_score, 'fpr' : false_positive_rate },
    y_true=materialized_view.y_true,
    y_pred=materialized_view.y_pred,
    sensitive_features=materialized_view.star_rating
)

In [6]:
fairness_metrics.overall

tpr    0.382556
fpr     0.02247
dtype: object

In [7]:
fairness_metrics.by_group

,tpr,fpr
star_rating,,
1,0.510244,0.076871
2,0.316556,0.050976
3,0.299492,0.029466
4,0.266754,0.019157
5,0.265413,0.008142
